In [38]:
from os import environ, path
import os
from pocketsphinx.pocketsphinx import *
from sphinxbase.sphinxbase import *

from pocketsphinx import DefaultConfig, Decoder, get_model_path, get_data_path

MODELDIR = get_model_path()
DATADIR = get_data_path()

from multiprocessing import Pool
import glob
import pickle

from tqdm import *

MODELDIR,DATADIR

('/Users/apple/anaconda2/lib/python2.7/site-packages/pocketsphinx/model',
 '/Users/apple/anaconda2/lib/python2.7/site-packages/pocketsphinx/data')

In [51]:
# Create a decoder with certain model
config = Decoder.default_config()
config.set_string('-hmm', os.path.join(MODELDIR, 'en-us-adapt'))
config.set_string('-allphone', os.path.join(MODELDIR, 'chess-phone.lm.bin'))
config.set_string('-dict', '/Users/chrischen/CS182/wechess-ai/chess-project.dic')
config.set_float('-lw', 2.0)
config.set_float('-beam', 1e-10)
config.set_float('-pbeam', 1e-10)

# Decode streaming data.
decoder = Decoder(config)

decoder.start_utt()
stream = open('/Users/chrischen/CS182/wechess-ai/wav/2_26.wav', 'rb')
i=0
while True:
    buf = stream.read(1024)
    if buf:
        decoder.process_raw(buf, False, False)
    else:
        break
decoder.end_utt()

hypothesis = decoder.hyp()
print ('Phonemes: ', [seg.word for seg in decoder.seg()])

RuntimeError: new_Decoder returned -1

In [52]:
def get_phonemes(file):
    # Decode streaming data
    decoder = Decoder(config)
    decoder.start_utt()
    stream = open(file, 'rb')
    i=0
    while True:
        buf = stream.read(1024)
        if buf:
            decoder.process_raw(buf, False, False)
        else:
            break
    decoder.end_utt()

    hypothesis = decoder.hyp()
    return [seg.word for seg in decoder.seg()]

In [53]:
get_phonemes("queen_1")

RuntimeError: new_Decoder returned -1

In [54]:
file_list = [i for i in glob.glob('/Users/chrischen/CS182/wechess-ai/wav/*') if '.wav' in i]

In [55]:
for i in tqdm(file_list):
    pass

0it [00:00, ?it/s]


In [56]:
phonemes = {
    'pawn': ['P', 'AO', 'N'],
    'knight': ['N', 'AY', 'T'],
    'bishop': ['B', 'IH', 'SH', 'AH', 'P'],
    'rook': ['R', 'UH', 'K'],
    'queen': ['K', 'W', 'IY', 'N'],
    'king': ['K', 'IH', 'NG'],
    'takes': ['T', 'EY', 'K', 'S'],
    'castle': ['K', 'AE', 'S', 'AH', 'L'],
    '1': ['W', 'AH', 'N'],
    '2': ['T', 'UW'],
    '3': ['TH', 'R', 'IY'],
    '4': ['F', 'AO', 'R'],
    '5': ['F', 'AY', 'V'],
    '6': ['S', 'IH', 'K', 'S'],
    '7': ['S', 'EH', 'V', 'AH', 'N'],
    '8': ['EY', 'T'],
    'a': ['EY'],
    'b': ['B', 'IY'],
    'c': ['S', 'IY'],
    'd': ['D', 'IY'],
    'e': ['IY'],
    'f': ['EH', 'F'],
    'g': ['JH', 'IY'],
    'h': ['EY', 'CH'],
    'kingside': ['K', 'IH', 'NG', 'S', 'AY', 'D'],
    'queenside': ['K', 'W', 'IY', 'N', 'S', 'AY', 'D'],
    'to': ['T', 'UW'],
}

In [ ]:
file_list[:4]

# DO NOT RUN THE CELL BELOW

In [57]:
# import phonemes of training sets
training_set = {}
for f in file_names:
    training_set[f] = []
    
# for each file name

def get_phoneme_pool(filepath):
    return (filepath.split('/wav/')[1].split('_')[0], get_phonemes(filepath))
    
pool_party = Pool(processes=2)

for res in tqdm(pool_party.imap_unordered(get_phoneme_pool, file_list)):
    training_set[res[0]].append(res[1])


    
# for name in file_names:
#     training_set[name] = []
#     # import phonemes for all 50 files with that file name
#     for i in range(1, 51):
#         training_set[name].append(get_phonemes(name + '_' + str(i)))

NameError: name 'file_names' is not defined

In [77]:
with open('training_set.pickle', 'wb') as handle:
    pickle.dump(training_set, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [78]:
piece_names = ['king', 'queen', 'knight', 'bishop', 'rook', 'pawn']
files = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h']
ranks = map(str, range(1,9))
actions = ['to', 'takes', 'castle', 'kingside', 'queenside']
indices_to_words = {}
words = piece_names + files + ranks + actions
for index, word in enumerate(words):
    indices_to_words[index] = (word, len(phonemes[word]))

In [79]:
indices_to_words

{0: ('king', 3),
 1: ('queen', 4),
 2: ('knight', 3),
 3: ('bishop', 5),
 4: ('rook', 3),
 5: ('pawn', 3),
 6: ('a', 1),
 7: ('b', 2),
 8: ('c', 2),
 9: ('d', 2),
 10: ('e', 1),
 11: ('f', 2),
 12: ('g', 2),
 13: ('h', 2),
 14: ('1', 3),
 15: ('2', 2),
 16: ('3', 3),
 17: ('4', 3),
 18: ('5', 3),
 19: ('6', 4),
 20: ('7', 5),
 21: ('8', 2),
 22: ('to', 2),
 23: ('takes', 4),
 24: ('castle', 5),
 25: ('kingside', 6),
 26: ('queenside', 7)}

In [80]:
transition_probabilities = [[0 for _ in range(27)] for _ in range(27)]

# set the transition probability of each word to itself to be n/n+1 where n is the number of 
# phonemes for the word
for index in range(27):
    _, n = indices_to_words[index]
    transition_probabilities[index][index] = n * 1.0 / (n + 1)

# pieces are indices 0-5 and always transition to a file aka letter (indices 6-13)
for i in range(6):
    _, n = indices_to_words[i]
    trans_prob = 1.0 / (n + 1)
    for j in range(6, 14):
        transition_probabilities[i][j] += trans_prob * 1.0 / 8

# letters are indices 6 - 13 and always transition to numbers (indices 14-21)
for i in range(6, 14):
    _, n = indices_to_words[i]
    trans_prob = 1.0 / (n + 1)
    for j in range(14, 22):
        transition_probabilities[i][j] += trans_prob * 1.0 / 8

# numbers are indices 14-21 and transition 50% of the time to actions
# (indices 22 - 23)
for i in range(14, 22):
    _, n = indices_to_words[i]
    trans_prob = 1.0 / (n + 1)
    for j in range(14, 22):    
        transition_probabilities[i][j] += trans_prob * 0.5 / 8
    for j in [22, 23]:
        transition_probabilities[i][j] += trans_prob * 0.5 / 2

# actions are indices 22-23 and always transition to letters (indices 6 - 13)
for i in [22, 23]:
    _, n = indices_to_words[i]
    trans_prob = 1.0 / (n + 1)
    for j in range(6, 14):
        transition_probabilities[i][j] += trans_prob * 1.0 / 8

# castling is index 24 and always transitions to (king/queen)side (indices 25-26)
trans_prob = 1.0 / 6
for j in [25, 26]:
    transition_probabilities[24][j] += trans_prob * 1.0 / 2

# (king/queen)side always transitions to (king/queen)side
for i in [25, 26]:
    trans_prob = 1.0 / (indices_to_words[i][1] + 1)
    transition_probabilities[i][i] += trans_prob

In [81]:
# get priors for pieces
priors = {}
priors['pawn'] = 8.0 / 17
for piece in ['king', 'queen', 'castle']:
    priors[piece] = 1.0 / 17
for piece in ['bishop', 'knight', 'rook']:
    priors[piece] = 2.0 / 17

In [82]:
# get list of phonemes
training_phonemes = set([item for sublist in training_set.values() for subsublist in sublist for item in subsublist])
actual_phonemes = set([item for sublist in phonemes.values() for item in sublist])
# actual_phonemes.add('SIL')

In [94]:
emission_model = {}
for phoneme in actual_phonemes:
    emission_model[phoneme] = {}
    for word in words:
        emission_model[phoneme][word] = 0.0

In [95]:
words = phonemes.keys()

In [96]:
for word, points in training_set.items():
    flattened = [item for point in points for item in point]
    total = 0
    for phoneme in actual_phonemes:
        phoneme_count = flattened.count(phoneme)
#         print phoneme
#         print word
        emission_model[phoneme][word] += phoneme_count
        total += phoneme_count
    for phoneme in actual_phonemes:
        emission_model[phoneme][word] /= 1.0 * total

In [97]:
for word in words:
    total = 0
    for phoneme in actual_phonemes:
        total += emission_model[phoneme][word]
    print total

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


In [87]:
# totals
totals = {}

# !!! Description of variables: !!!

In [88]:
# dictionary mapping words to lists of phonemes returned by sphinx
# check the group chat for the relevant pickle file
with open('training_set.pickle', 'rb') as handle:
    training_set = pickle.load(handle)

In [90]:
# prior marginal probability for each piece
priors

{'bishop': 0.11764705882352941,
 'castle': 0.058823529411764705,
 'king': 0.058823529411764705,
 'knight': 0.11764705882352941,
 'pawn': 0.47058823529411764,
 'queen': 0.058823529411764705,
 'rook': 0.11764705882352941}

In [91]:
# transition probabilities from every word to every other word
# note that we can divide the words into the following categories: 
    # piece, letter, num, action, castle, side
# and that the only two valid sequences for a command are:
    # (1) piece --> letter --> number --> action --> letter --> number
        # e.g., pawn e2 to e4
    # (2) castle --> side
        # e.g., "castle kingside"
transition_probabilities

[[0.75,
  0,
  0,
  0,
  0,
  0,
  0.03125,
  0.03125,
  0.03125,
  0.03125,
  0.03125,
  0.03125,
  0.03125,
  0.03125,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0,
  0.8,
  0,
  0,
  0,
  0,
  0.025,
  0.025,
  0.025,
  0.025,
  0.025,
  0.025,
  0.025,
  0.025,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0,
  0,
  0.75,
  0,
  0,
  0,
  0.03125,
  0.03125,
  0.03125,
  0.03125,
  0.03125,
  0.03125,
  0.03125,
  0.03125,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0,
  0,
  0,
  0.8333333333333334,
  0,
  0,
  0.020833333333333332,
  0.020833333333333332,
  0.020833333333333332,
  0.020833333333333332,
  0.020833333333333332,
  0.020833333333333332,
  0.020833333333333332,
  0.020833333333333332,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0,
  0,
  0,
  0,
  0.75,
  0,
  0.03125,
  0.03125,
  0.03125,
  0.03125,
  0.03125,
  0.03125,
  0.03125,
  0.03125,
  0,
  0,
  0,
  0,
  0,
  0,


In [92]:
# tells you what word each index in transition_probabilities corresponds to
# and how many phonemes that word has
indices_to_words

{0: ('king', 3),
 1: ('queen', 4),
 2: ('knight', 3),
 3: ('bishop', 5),
 4: ('rook', 3),
 5: ('pawn', 3),
 6: ('a', 1),
 7: ('b', 2),
 8: ('c', 2),
 9: ('d', 2),
 10: ('e', 1),
 11: ('f', 2),
 12: ('g', 2),
 13: ('h', 2),
 14: ('1', 3),
 15: ('2', 2),
 16: ('3', 3),
 17: ('4', 3),
 18: ('5', 3),
 19: ('6', 4),
 20: ('7', 5),
 21: ('8', 2),
 22: ('to', 2),
 23: ('takes', 4),
 24: ('castle', 5),
 25: ('kingside', 6),
 26: ('queenside', 7)}

In [126]:
indices_to_words

{0: ('king', 3),
 1: ('queen', 4),
 2: ('knight', 3),
 3: ('bishop', 5),
 4: ('rook', 3),
 5: ('pawn', 3),
 6: ('a', 1),
 7: ('b', 2),
 8: ('c', 2),
 9: ('d', 2),
 10: ('e', 1),
 11: ('f', 2),
 12: ('g', 2),
 13: ('h', 2),
 14: ('1', 3),
 15: ('2', 2),
 16: ('3', 3),
 17: ('4', 3),
 18: ('5', 3),
 19: ('6', 4),
 20: ('7', 5),
 21: ('8', 2),
 22: ('to', 2),
 23: ('takes', 4),
 24: ('castle', 5),
 25: ('kingside', 6),
 26: ('queenside', 7)}

In [127]:
words_to_indices = {val[0]:key for key,val in indices_to_words.iteritems()}

In [128]:
words_to_indices

{'1': 14,
 '2': 15,
 '3': 16,
 '4': 17,
 '5': 18,
 '6': 19,
 '7': 20,
 '8': 21,
 'a': 6,
 'b': 7,
 'bishop': 3,
 'c': 8,
 'castle': 24,
 'd': 9,
 'e': 10,
 'f': 11,
 'g': 12,
 'h': 13,
 'king': 0,
 'kingside': 25,
 'knight': 2,
 'pawn': 5,
 'queen': 1,
 'queenside': 26,
 'rook': 4,
 'takes': 23,
 'to': 22}

In [106]:
# Dictionary mapping phonemes to sub-dictionaries
    # sub-dictionaries map words to P(phoneme|word)
emission_model

{'AE': {'1': 0.0,
  '2': 0.0,
  '3': 0.0,
  '4': 0.0,
  '5': 0.0,
  '6': 0.0,
  '7': 0.0,
  '8': 0.0,
  'a': 0.0,
  'b': 0.0,
  'bishop': 0.0,
  'c': 0.0,
  'castle': 0.0,
  'd': 0.0,
  'e': 0.0,
  'f': 0.0,
  'g': 0.0,
  'h': 0.0,
  'king': 0.0,
  'kingside': 0.0,
  'knight': 0.0,
  'pawn': 0.0,
  'queen': 0.0,
  'queenside': 0.0,
  'rook': 0.0,
  'takes': 0.0,
  'to': 0.0},
 'AH': {'1': 0.0,
  '2': 0.0,
  '3': 0.0,
  '4': 0.0,
  '5': 0.0,
  '6': 0.0,
  '7': 0.0,
  '8': 0.0,
  'a': 0.0,
  'b': 0.0,
  'bishop': 0.0,
  'c': 0.0,
  'castle': 0.0,
  'd': 0.0,
  'e': 0.0,
  'f': 0.0,
  'g': 0.0,
  'h': 0.0,
  'king': 0.0,
  'kingside': 0.0,
  'knight': 0.0,
  'pawn': 0.0,
  'queen': 0.0,
  'queenside': 0.0,
  'rook': 0.0,
  'takes': 0.0,
  'to': 0.0},
 'AO': {'1': 0.0,
  '2': 0.0,
  '3': 0.0,
  '4': 0.0,
  '5': 0.0,
  '6': 0.0,
  '7': 0.0,
  '8': 0.0,
  'a': 0.0,
  'b': 0.0,
  'bishop': 0.0,
  'c': 0.0,
  'castle': 0.0,
  'd': 0.0,
  'e': 0.0,
  'f': 0.0,
  'g': 0.0,
  'h': 0.0,
  'king': 

In [107]:
sample_input = ['EY', 'W', 'AO', 'AH', 'AY', 'AH', 'UW', \
                'F', 'SIL', 'V', 'N', 'T', 'S', 'IH', 'T', 'IH', 'K', 'S', 'S', 'T', \
                'TH', 'OW', 'JH', 'IY', 'IH', 'UW', 'SIL']

In [125]:
transition_probabilities

[[0.75,
  0,
  0,
  0,
  0,
  0,
  0.03125,
  0.03125,
  0.03125,
  0.03125,
  0.03125,
  0.03125,
  0.03125,
  0.03125,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0,
  0.8,
  0,
  0,
  0,
  0,
  0.025,
  0.025,
  0.025,
  0.025,
  0.025,
  0.025,
  0.025,
  0.025,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0,
  0,
  0.75,
  0,
  0,
  0,
  0.03125,
  0.03125,
  0.03125,
  0.03125,
  0.03125,
  0.03125,
  0.03125,
  0.03125,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0,
  0,
  0,
  0.8333333333333334,
  0,
  0,
  0.020833333333333332,
  0.020833333333333332,
  0.020833333333333332,
  0.020833333333333332,
  0.020833333333333332,
  0.020833333333333332,
  0.020833333333333332,
  0.020833333333333332,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0,
  0,
  0,
  0,
  0.75,
  0,
  0.03125,
  0.03125,
  0.03125,
  0.03125,
  0.03125,
  0.03125,
  0.03125,
  0.03125,
  0,
  0,
  0,
  0,
  0,
  0,


In [123]:
words = phonemes.keys()

In [130]:
w_to_i = {val[0]:key for key,val in indices_to_words.iteritems()}

In [131]:
from collections import Counter

In [134]:
emission_model

{'AE': {'1': 0.0033444816053511705,
  '2': 0.0,
  '3': 0.0,
  '4': 0.0,
  '5': 0.020491803278688523,
  '6': 0.006666666666666667,
  '7': 0.0,
  '8': 0.0,
  'a': 0.0,
  'b': 0.0,
  'bishop': 0.0,
  'c': 0.0,
  'castle': 0.0,
  'd': 0.0,
  'e': 0.0,
  'f': 0.0,
  'g': 0.0,
  'h': 0.0,
  'king': 0.0,
  'kingside': 0.03130755064456722,
  'knight': 0.0,
  'pawn': 0.0038022813688212928,
  'queen': 0.0,
  'queenside': 0.0,
  'rook': 0.0,
  'takes': 0.009852216748768473,
  'to': 0.0},
 'AH': {'1': 0.2140468227424749,
  '2': 0.005434782608695652,
  '3': 0.03859649122807018,
  '4': 0.06201550387596899,
  '5': 0.06147540983606557,
  '6': 0.03111111111111111,
  '7': 0.1585677749360614,
  '8': 0.0,
  'a': 0.0,
  'b': 0.0,
  'bishop': 0.21580547112462006,
  'c': 0.0,
  'castle': 0.0,
  'd': 0.0,
  'e': 0.0,
  'f': 0.0,
  'g': 0.0,
  'h': 0.0,
  'king': 0.0,
  'kingside': 0.07734806629834254,
  'knight': 0.0,
  'pawn': 0.10266159695817491,
  'queen': 0.0031446540880503146,
  'queenside': 0.0,
  'rook

In [132]:
def viterbi_update(sample_input):
    # initialization
    ml_sequence = []
    v_scores = Counter()
    for key,val in priors.iteritems():
        v_scores[key] = val
    for ph in sample_input:
        assert(ph in emission_model.keys())
        new_scores = Counter()
        emission = emission_model[ph]
        for word in words:
            tmp_score = max([transition_probabilities[w_to_i[word_p]][w_to_i[word]] \
                             *v_scores[word_p] for word_p in words])
            new_scores[word] = tmp_score * emission[word]
        new_scores.normalize()
        most_likely = max(new_scores, key=new_scores.get)
        ml_sequence.append(most_likely)
        v_scores = new_scores
    assert(len(ml_sequence) == len(sample_input))
    return ml_sequence

In [133]:
viterbi_update(sample_input)

AssertionError: 

In [108]:
v_scores = priors

In [122]:
priors

{'bishop': 0.11764705882352941,
 'castle': 0.058823529411764705,
 'king': 0.058823529411764705,
 'knight': 0.11764705882352941,
 'pawn': 0.47058823529411764,
 'queen': 0.058823529411764705,
 'rook': 0.11764705882352941}

## Duplicate data

In [ ]:
import glob, os

In [ ]:
def rename(dir, pattern, titlePattern):
    for pathAndFilename in glob.iglob(os.path.join(dir, pattern)):
        title, ext = os.path.splitext(os.path.basename(pathAndFilename))
        print(os.path.join(dir, titlePattern % title + ext))
        os.rename(pathAndFilename, 
                  os.path.join(dir, titlePattern % title + ext))

In [ ]:
rename(r'/Users/apple/Desktop/16-17/CS182/chess-project/wav2', r'*.wav', r'%sb')

### SOPHIE

In [ ]:
#!/usr/bin/env python

from pocketsphinx.pocketsphinx import *
from sphinxbase.sphinxbase import *

import os
import pyaudio
import wave
import audioop
from collections import deque
import time
import math

from pocketsphinx import DefaultConfig, Decoder, get_model_path, get_data_path


"""
Written by Sophie Li, 2016
http://blog.justsophie.com/python-speech-to-text-with-pocketsphinx/
"""

class SpeechDetector:
    def __init__(self):
        # Microphone stream config.
        self.CHUNK = 1024  # CHUNKS of bytes to read each time from mic
        self.FORMAT = pyaudio.paInt16
        self.CHANNELS = 1
        self.RATE = 16000

        self.SILENCE_LIMIT = 1  # Silence limit in seconds. The max ammount of seconds where
                           # only silence is recorded. When this time passes the
                           # recording finishes and the file is decoded

        self.PREV_AUDIO = 0.5  # Previous audio (in seconds) to prepend. When noise
                          # is detected, how much of previously recorded audio is
                          # prepended. This helps to prevent chopping the beginning
                          # of the phrase.

        self.THRESHOLD = 4500
        self.num_phrases = -1

        # These will need to be modified according to where the pocketsphinx folder is
        MODELDIR = get_model_path()
        DATADIR = get_data_path()

        # Create a decoder with certain model
        config = Decoder.default_config()
        config.set_string('-hmm', os.path.join(MODELDIR, 'en-us'))
#         config.set_string('-lm', os.path.join(MODELDIR, 'en-us.lm.bin'))
#         config.set_string('-dict', os.path.join(MODELDIR, 'cmudict-en-us.dict'))
        config.set_string('-allphone', os.path.join(MODELDIR, 'chess-phone.lm.bin'))
#         config.set_float('-lw', 2.0)
#         config.set_float('-beam', 1e-10)
#         config.set_float('-pbeam', 1e-10)

        # Creaders decoder object for streaming data.
        self.decoder = Decoder(config)

    def setup_mic(self, num_samples=50):
        """ Gets average audio intensity of your mic sound. You can use it to get
            average intensities while you're talking and/or silent. The average
            is the avg of the .2 of the largest intensities recorded.
        """
        print "Getting intensity values from mic."
        p = pyaudio.PyAudio()
        stream = p.open(format=self.FORMAT, 
                        channels=self.CHANNELS,
                        rate=self.RATE, 
                        input=True, 
                        frames_per_buffer=self.CHUNK)

        values = [math.sqrt(abs(audioop.avg(stream.read(self.CHUNK), 4)))
                  for x in range(num_samples)]
        values = sorted(values, reverse=True)
        r = sum(values[:int(num_samples * 0.2)]) / int(num_samples * 0.2)
        print " Finished "
        print " Average audio intensity is ", r
        stream.close()
        p.terminate()

        if r < 3000:
            self.THRESHOLD = 3500
        else:
            self.THRESHOLD = r + 100

    def save_speech(self, data, p):
        """
        Saves mic data to temporary WAV file. Returns filename of saved
        file
        """
        filename = 'output_'+str(int(time.time()))
        # writes data to WAV file
        data = ''.join(data)
        wf = wave.open(filename + '.wav', 'wb')
        print('save_speech...')
        wf.setnchannels(1)
        wf.setsampwidth(p.get_sample_size(pyaudio.paInt16))
        wf.setframerate(16000)  # TODO make this value a function parameter?
        wf.writeframes(data)
        wf.close()
        return filename + '.wav'

    def decode_phrase(self, wav_file):
        self.decoder.start_utt()
        stream = open(wav_file, "rb")
        while True:
            buf = stream.read(1024)
            if buf:
                self.decoder.process_raw(buf, False, False)
            else:
                break
        self.decoder.end_utt()
        words = []
        [words.append(seg.word) for seg in self.decoder.seg()]
        return words

    def run(self):
        """
        Listens to Microphone, extracts phrases from it and calls pocketsphinx
        to decode the sound
        """
        self.setup_mic()

        #Open stream
        p = pyaudio.PyAudio()
        stream = p.open(format=self.FORMAT, 
                        channels=self.CHANNELS, 
                        rate=self.RATE, 
                        input=True, 
                        frames_per_buffer=self.CHUNK)
        print "* Mic set up and listening. "

        audio2send = []
        cur_data = ''  # current chunk of audio data
        rel = self.RATE/self.CHUNK
        slid_win = deque(maxlen=self.SILENCE_LIMIT * rel)
        #Prepend audio from 0.5 seconds before noise was detected
        prev_audio = deque(maxlen=self.PREV_AUDIO * rel)
        started = False

        while True:
            cur_data = stream.read(self.CHUNK)
            slid_win.append(math.sqrt(abs(audioop.avg(cur_data, 4))))
            
            thresh = sum([x > self.THRESHOLD for x in slid_win])
#             print 'threshold:{thresh}'.format(thresh=thresh)
            if thresh > 0:
                if started == False:
                    print "Starting recording of phrase"
                    started = True
                audio2send.append(cur_data)

            elif started:
                print "Finished recording, decoding phrase"
                filename = self.save_speech(list(prev_audio) + audio2send, p)
                r = self.decode_phrase(filename)
                print "DETECTED: ", r

                # Removes temp audio file
                os.remove(filename)
                # Reset all
                started = False
                slid_win = deque(maxlen=self.SILENCE_LIMIT * rel)
                prev_audio = deque(maxlen=0.5 * rel)
                audio2send = []
                print "Listening ..."

            else:
                prev_audio.append(cur_data)

        print "* Done listening"
        stream.close()
        p.terminate()

In [ ]:
sd = SpeechDetector()
sd.run()

In [ ]:
pwd

In [100]:
MODELDIR

'/Users/apple/anaconda2/lib/python2.7/site-packages/pocketsphinx/model'

In [102]:
with open('training_set.pickle', 'wb') as handle:
    pickle.dump(training_set, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [118]:
"""
Contains functions needed to generate and traverse the HMM
"""

import glob
import pickle
import os

from multiprocessing import Pool
from tqdm import *

from pocketsphinx import DefaultConfig, Decoder, get_model_path, get_data_path
from pocketsphinx.pocketsphinx import *
from sphinxbase.sphinxbase import *

MODELDIR = get_model_path()
DATADIR = get_data_path()


config = Decoder.default_config()
config.set_string('-hmm', os.path.join(MODELDIR, 'en-us-adapt'))
config.set_string('-allphone', os.path.join(MODELDIR, 'chess-phone.lm.bin'))
config.set_string('-dict', '/Users/apple/Desktop/16-17/CS182/wechess-ai/chess-project.dic')
config.set_float('-lw', 2.0)
config.set_float('-beam', 1e-10)
config.set_float('-pbeam', 1e-10)

# Decode streaming data.
decoder = Decoder(config)


def get_phonemes(file):
    # Decode streaming data
    decoder = Decoder(config)
    decoder.start_utt()
    stream = open(file, 'rb')
    i=0
    while True:
        buf = stream.read(1024)
        if buf:
            decoder.process_raw(buf, False, False)
        else:
            break
    decoder.end_utt()

    hypothesis = decoder.hyp()
    return [seg.word for seg in decoder.seg()]

file_list = [i for i in glob.glob('wav/*') if '.wav' in i]

phonemes = {
    'pawn': ['P', 'AO', 'N'],
    'knight': ['N', 'AY', 'T'],
    'bishop': ['B', 'IH', 'SH', 'AH', 'P'],
    'rook': ['R', 'UH', 'K'],
    'queen': ['K', 'W', 'IY', 'N'],
    'king': ['K', 'IH', 'NG'],
    'takes': ['T', 'EY', 'K', 'S'],
    'castle': ['K', 'AE', 'S', 'AH', 'L'],
    '1': ['W', 'AH', 'N'],
    '2': ['T', 'UW'],
    '3': ['TH', 'R', 'IY'],
    '4': ['F', 'AO', 'R'],
    '5': ['F', 'AY', 'V'],
    '6': ['S', 'IH', 'K', 'S'],
    '7': ['S', 'EH', 'V', 'AH', 'N'],
    '8': ['EY', 'T'],
    'a': ['EY'],
    'b': ['B', 'IY'],
    'c': ['S', 'IY'],
    'd': ['D', 'IY'],
    'e': ['IY'],
    'f': ['EH', 'F'],
    'g': ['JH', 'IY'],
    'h': ['EY', 'CH'],
    'kingside': ['K', 'IH', 'NG', 'S', 'AY', 'D'],
    'queenside': ['K', 'W', 'IY', 'N', 'S', 'AY', 'D'],
    'to': ['T', 'UW'],
}


with open('training_set.pickle', 'rb') as handle:
    training_set = pickle.load(handle)


piece_names = ['king', 'queen', 'knight', 'bishop', 'rook', 'pawn']
files = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h']
ranks = map(str, range(1,9))
actions = ['to', 'takes', 'castle', 'kingside', 'queenside']
indices_to_words = {}
words = piece_names + files + ranks + actions
for index, word in enumerate(words):
    indices_to_words[index] = (word, len(phonemes[word]))

# # import phonemes of training sets
# training_set = {}
# for w in words:
#     training_set[w] = []

# for each file name

def get_phoneme_pool(filepath):
    return (filepath.split('/wav/')[1].split('_')[0], get_phonemes(filepath))

pool_party = Pool(processes=2)

for res in tqdm(pool_party.imap_unordered(get_phoneme_pool, file_list)):
    training_set[res[0]].append(res[1])

transition_probabilities = [[0 for _ in range(27)] for _ in range(27)]

# set the transition probability of each word to itself to be n/n+1 where n is the number of
# phonemes for the word
for index in range(27):
    _, n = indices_to_words[index]
    transition_probabilities[index][index] = n * 1.0 / (n + 1)

# pieces are indices 0-5 and always transition to a file aka letter (indices 6-13)
for i in range(6):
    _, n = indices_to_words[i]
    trans_prob = 1.0 / (n + 1)
    for j in range(6, 14):
        transition_probabilities[i][j] += trans_prob * 1.0 / 8

# letters are indices 6 - 13 and always transition to numbers (indices 14-21)
for i in range(6, 14):
    _, n = indices_to_words[i]
    trans_prob = 1.0 / (n + 1)
    for j in range(14, 22):
        transition_probabilities[i][j] += trans_prob * 1.0 / 8

# numbers are indices 14-21 and transition 50% of the time to actions
# (indices 22 - 23)
for i in range(14, 22):
    _, n = indices_to_words[i]
    trans_prob = 1.0 / (n + 1)
    for j in range(14, 22):
        transition_probabilities[i][j] += trans_prob * 0.5 / 8
    for j in [22, 23]:
        transition_probabilities[i][j] += trans_prob * 0.5 / 2

# actions are indices 22-23 and always transition to letters (indices 6 - 13)
for i in [22, 23]:
    _, n = indices_to_words[i]
    trans_prob = 1.0 / (n + 1)
    for j in range(6, 14):
        transition_probabilities[i][j] += trans_prob * 1.0 / 8

# castling is index 24 and always transitions to (king/queen)side (indices 25-26)
trans_prob = 1.0 / 6
for j in [25, 26]:
    transition_probabilities[24][j] += trans_prob * 1.0 / 2

# (king/queen)side always transitions to (king/queen)side
for i in [25, 26]:
    trans_prob = 1.0 / (indices_to_words[i][1] + 1)
    transition_probabilities[i][i] += trans_prob

# get priors for pieces
priors = {}
priors['pawn'] = 8.0 / 17
for piece in ['king', 'queen', 'castle']:
    priors[piece] = 1.0 / 17
for piece in ['bishop', 'knight', 'rook']:
    priors[piece] = 2.0 / 17

# get list of phonemes
training_phonemes = set([item for sublist in training_set.values() for subsublist in sublist for item in subsublist])
actual_phonemes = set([item for sublist in phonemes.values() for item in sublist])
# actual_phonemes.add('SIL')

emission_model = {}
for phoneme in actual_phonemes:
    emission_model[phoneme] = {}
    for word in words:
        emission_model[phoneme][word] = 0.0

words = phonemes.keys()

for word, points in training_set.items():
    flattened = [item for point in points for item in point]
    total = 0
    for phoneme in actual_phonemes:
        phoneme_count = flattened.count(phoneme)
#         print phoneme
#         print word
        emission_model[phoneme][word] += phoneme_count
        total += phoneme_count
    for phoneme in actual_phonemes:
        emission_model[phoneme][word] /= 1.0 * total

#TODO @Chris maybe move this elsewhere
# transition probabilities from every word to every other word
# note that we can divide the words into the following categories:
    # piece, letter, num, action, castle, side
# and that the only two valid sequences for a command are:
    # (1) piece --> letter --> number --> action --> letter --> number
        # e.g., pawn e2 to e4
    # (2) castle --> side
        # e.g., "castle kingside"

0it [00:00, ?it/s]


KeyError: '9'

In [120]:
training_set.pop('9')

[]

In [121]:
emission_model

{'AE': {'1': 0.0033444816053511705,
  '2': 0.0,
  '3': 0.0,
  '4': 0.0,
  '5': 0.020491803278688523,
  '6': 0.006666666666666667,
  '7': 0.0,
  '8': 0.0,
  'a': 0.0,
  'b': 0.0,
  'bishop': 0.0,
  'c': 0.0,
  'castle': 0.0,
  'd': 0.0,
  'e': 0.0,
  'f': 0.0,
  'g': 0.0,
  'h': 0.0,
  'king': 0.0,
  'kingside': 0.03130755064456722,
  'knight': 0.0,
  'pawn': 0.0038022813688212928,
  'queen': 0.0,
  'queenside': 0.0,
  'rook': 0.0,
  'takes': 0.009852216748768473,
  'to': 0.0},
 'AH': {'1': 0.2140468227424749,
  '2': 0.005434782608695652,
  '3': 0.03859649122807018,
  '4': 0.06201550387596899,
  '5': 0.06147540983606557,
  '6': 0.03111111111111111,
  '7': 0.1585677749360614,
  '8': 0.0,
  'a': 0.0,
  'b': 0.0,
  'bishop': 0.21580547112462006,
  'c': 0.0,
  'castle': 0.0,
  'd': 0.0,
  'e': 0.0,
  'f': 0.0,
  'g': 0.0,
  'h': 0.0,
  'king': 0.0,
  'kingside': 0.07734806629834254,
  'knight': 0.0,
  'pawn': 0.10266159695817491,
  'queen': 0.0031446540880503146,
  'queenside': 0.0,
  'rook